In [74]:
import requests
import re
from bs4 import BeautifulSoup

jobs={}
seen_jobs={}
for i in range(0,51,25):
    base_url = f"https://www.linkedin.com/jobs/search?keywords=software%20engineer%20OR%20engineering%20manager&location=United%20States&pageNum=0&start={i}"

    #target_roles=['Software Engineer','Engineering Manager']
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        job_listings = soup.find_all('div', {'class':'job-search-card'})
        for job in job_listings:
            try:
                date = job.find('time', {'class':'job-search-card__listdate'}).attrs['datetime']
            except AttributeError:
                date = job.find('time', {'class':'job-search-card__listdate--new'}).attrs['datetime']
            title = job.find('h3', {'class': 'base-search-card__title'}).text.strip()
            company = job.find('a', {'class': 'hidden-nested-link'}).text.strip()
            location = job.find('span', {'class': 'job-search-card__location'}).text.strip()
            link = job.find('a', {'class': 'base-card__full-link'}).attrs['href']
            pattern = r"(.*-)(\d+)"
            sanitized_link = re.search(pattern,link).group(2)

            #check cache; skip if already seen
            if seen_jobs.get(sanitized_link,'') != '':
                #print(f"we've seen {sanitized_link}")
                continue
            else:
                seen_jobs[sanitized_link]=True
                
            #test
#             if company != 'Airbnb':
#                 continue
#             print(f"title:{title}, id:{sanitized_link}, date:{date}")
    
            #initialize kv entry
            if company not in jobs:
                jobs.setdefault(company,{})
                jobs[company]={'eng':0,'em':0,'maxDate':''}
                
            #increment count
            if 'Manager' not in title:
                jobs[company]['eng']=jobs.get(company,{}).get('eng',0)+1
            else:
                jobs[company]['em']=jobs.get(company,{}).get('em',0)+1
                
            #set max date
            jobs[company]['maxDate'] = max(jobs[company]['maxDate'], date)

    else:
        print("Failed to fetch job listings.")
        break
print(f"jobs found:{len(jobs)}")

jobs=dict(sorted(jobs.items(),key=lambda x:x[1]['eng']+x[1]['em'], reverse=True))
filtered_jobs={k:v for k,v in jobs.items() if v['eng']+v['em']>1}
print('\n'.join([f"{key}: {value}" for key, value in filtered_jobs.items()]))
#print('\n'.join([f"{key}: {value}" for key, value in seen_jobs.items()]))

Failed to fetch job listings.
jobs found:45
Ever: {'eng': 3, 'em': 0, 'maxDate': '2024-05-16'}
Ashby: {'eng': 0, 'em': 3, 'maxDate': '2024-05-01'}
Airbnb: {'eng': 2, 'em': 0, 'maxDate': '2024-05-16'}
LinkedIn: {'eng': 2, 'em': 0, 'maxDate': '2024-05-14'}
Open Systems Inc.: {'eng': 2, 'em': 0, 'maxDate': '2024-05-15'}
Netflix: {'eng': 2, 'em': 0, 'maxDate': '2024-05-09'}
Nike: {'eng': 2, 'em': 0, 'maxDate': '2024-05-13'}
Midpoint Markets: {'eng': 2, 'em': 0, 'maxDate': '2024-05-11'}
Tagup, Inc.: {'eng': 2, 'em': 0, 'maxDate': '2024-05-10'}
Saragossa: {'eng': 2, 'em': 0, 'maxDate': '2024-05-13'}
Microsoft: {'eng': 2, 'em': 0, 'maxDate': '2024-05-03'}
Slack: {'eng': 2, 'em': 0, 'maxDate': '2024-05-13'}
